In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("npl").getOrCreate()

25/01/11 21:23:07 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
data = spark.read.csv("smsspamcollection/SMSSpamCollection",inferSchema=True, sep='\t')  

In [4]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1', 'text')

In [5]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [6]:
from pyspark.sql.functions import length

In [7]:
data = data.withColumn('length', length(data['text']))

In [8]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [9]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [10]:
from pyspark.ml.feature import Tokenizer, CountVectorizer, StringIndexer, IDF, StopWordsRemover

In [11]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec',outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

In [12]:
from pyspark.ml.feature import VectorAssembler

In [13]:
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

In [36]:
from pyspark.ml.classification import LogisticRegression

In [37]:
nb = LogisticRegression()

In [38]:
from pyspark.ml import Pipeline

In [39]:
data_prep_pipe = Pipeline(stages = [ham_spam_to_numeric, tokenizer, stop_remove, count_vec, idf, clean_up])

In [40]:
cleaner = data_prep_pipe.fit(data)

In [41]:
cleaner_data = cleaner.transform(data)

In [42]:
cleaner_data = cleaner_data.select('label', 'features')

In [43]:
cleaner_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [44]:
training, test = cleaner_data.randomSplit([0.7, 0.3])

In [45]:
spam_detector = nb.fit(training)

25/01/11 22:28:01 WARN DAGScheduler: Broadcasting large task binary with size 1166.0 KiB
25/01/11 22:28:02 WARN DAGScheduler: Broadcasting large task binary with size 1166.7 KiB
25/01/11 22:28:03 WARN DAGScheduler: Broadcasting large task binary with size 1166.7 KiB
25/01/11 22:28:03 WARN DAGScheduler: Broadcasting large task binary with size 1166.7 KiB
25/01/11 22:28:03 WARN DAGScheduler: Broadcasting large task binary with size 1166.7 KiB
25/01/11 22:28:04 WARN DAGScheduler: Broadcasting large task binary with size 1166.7 KiB
25/01/11 22:28:04 WARN DAGScheduler: Broadcasting large task binary with size 1166.7 KiB
25/01/11 22:28:04 WARN DAGScheduler: Broadcasting large task binary with size 1166.7 KiB
25/01/11 22:28:04 WARN DAGScheduler: Broadcasting large task binary with size 1166.7 KiB
25/01/11 22:28:05 WARN DAGScheduler: Broadcasting large task binary with size 1166.7 KiB
25/01/11 22:28:05 WARN DAGScheduler: Broadcasting large task binary with size 1166.7 KiB
25/01/11 22:28:05 WAR

In [46]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [47]:
test_result = spam_detector.transform(test)

In [48]:
test_result.show()

25/01/11 22:28:22 WARN DAGScheduler: Broadcasting large task binary with size 1271.8 KiB


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[30.4126101020943...|[0.99999999999993...|       0.0|
|  0.0|(13424,[0,1,2,13,...|[15.0998324530106...|[0.99999972316183...|       0.0|
|  0.0|(13424,[0,1,2,41,...|[16.2716385279227...|[0.99999991423360...|       0.0|
|  0.0|(13424,[0,1,4,50,...|[25.2266670493935...|[0.99999999998892...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[14.0141204135137...|[0.99999918013097...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[14.0918298910027...|[0.99999924142992...|       0.0|
|  0.0|(13424,[0,1,11,32...|[17.2027042355489...|[0.99999996619659...|       0.0|
|  0.0|(13424,[0,1,14,18...|[30.3794909260088...|[0.99999999999993...|       0.0|
|  0.0|(13424,[0,1,20,27...|[16.0744606613038...|[0.99999989553991...|       0.0|
|  0.0|(13424,[0

In [49]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [50]:
acc_eval = MulticlassClassificationEvaluator()

In [51]:
acc = acc_eval.evaluate(test_result)

25/01/11 22:28:31 WARN DAGScheduler: Broadcasting large task binary with size 1276.5 KiB


In [52]:
acc

0.9789845758541135